![ngo_project_image](ngo_project_image.jpg)

GoodThought NGO has been a catalyst for positive change, focusing its efforts on education, healthcare, and sustainable development to make a significant difference in communities worldwide. With this mission, GoodThought has orchestrated an array of assignments aimed at uplifting underprivileged populations and fostering long-term growth.

This project offers a hands-on opportunity to explore how data-driven insights can direct and enhance these humanitarian efforts. In this project, you'll engage with the GoodThought PostgreSQL database, which encapsulates detailed records of assignments, funding, impacts, and donor activities from 2010 to 2023. This comprehensive dataset includes:

- **`Assignments`:** Details about each project, including its name, duration (start and end dates), budget, geographical region, and the impact score.
- **`Donations`:** Records of financial contributions, linked to specific donors and assignments, highlighting how financial support is allocated and utilized.
- **`Donors`:** Information on individuals and organizations that fund GoodThought’s projects, including donor types.

Refer to the below ERD diagram for a visual representation of the relationships between these data tables:
<img src="erd.png" alt="ERD" width="50%" height="50%">


You will execute SQL queries to answer two questions, as listed in the instructions. Good luck!


In [27]:
-- highest_donation_assignments

-- Create CTE to combine the donations and donors tables
WITH donation_summary AS (
	SELECT public.donors.donor_type,
			public.donations.assignment_id,
			SUM(public.donations.amount) AS amount2
		FROM public.donations
		LEFT JOIN public.donors
		ON public.donations.donor_id = public.donors.donor_id
		GROUP BY public.donations.assignment_id,public.donors.donor_type
		
)

SELECT A.assignment_name,
		A.region,
		ROUND(D.amount2,2) AS rounded_total_donation_amount,
		D.donor_type
	FROM public.assignments AS A
	JOIN donation_summary AS D
	ON A.assignment_id = D.assignment_id
	ORDER BY rounded_total_donation_amount DESC
	LIMIT 5;


,assignment_name,region,rounded_total_donation_amount,donor_type
0,Assignment_3033,East,3840.66,Individual
1,Assignment_300,West,3133.98,Organization
2,Assignment_4114,North,2778.57,Organization
3,Assignment_1765,West,2626.98,Organization
4,Assignment_268,East,2488.69,Individual


In [28]:
-- top_regional_impact_assignments

-- Create CTE to combine the donations and donors tables
WITH donation_summary AS (
	SELECT public.donations.assignment_id,
			SUM(public.donations.amount) AS num_total_donations
		FROM public.donations
		LEFT JOIN public.donors
		ON public.donations.donor_id = public.donors.donor_id
		GROUP BY public.donations.assignment_id
),

assignment_summary AS (
	SELECT public.assignments.assignment_id,
			public.assignments.assignment_name,
			public.assignments.region,
			public.assignments.impact_score,
		ROW_NUMBER() OVER (PARTITION BY public.assignments.region ORDER BY public.assignments.impact_score DESC) AS rank
	FROM public.assignments
)

SELECT ASUM.assignment_name, ASUM.region, ASUM.impact_score, DS.num_total_donations
	FROM assignment_summary AS ASUM
	JOIN donation_summary AS DS
	ON ASUM.assignment_id = DS.assignment_id
	WHERE ASUM.rank = 1
	ORDER BY region, ASUM.impact_score DESC

,assignment_name,region,impact_score,num_total_donations
0,Assignment_316,East,10.00,530.81
1,Assignment_2253,North,9.99,872.03
2,Assignment_3547,South,10.00,880.15
3,Assignment_3764,West,9.99,716.72
